In [3]:
'''
To test real-time bandwidth between two nodes:
step1: install 'iperf3' on testing nodes
step2: run on node (server) terminal: ~$ iperf3 -s
step3: run on the other node (client) termnial: ~$ iperf3 -c [server's IP address]

'''

import paramiko
import time

def limit_bandwidth(ssh, limit, duration, interface='eth0'):
    # Calculate the bandwidth based on the percentage of a full bandwidth assumption (e.g., 1000Mbps)
    full_bandwidth = 16*1000  # Full bandwidth in Mbps
    limit_bandwidth = full_bandwidth * limit / 100
    
    # Commands to setup bandwidth control
    setup_cmds = [
        f"sudo tc qdisc add dev {interface} root handle 1: htb default 11",
        f"sudo tc class add dev {interface} parent 1: classid 1:1 htb rate {limit_bandwidth}mbit"
    ]
    
    # Command to remove bandwidth control
    remove_cmd = f"sudo tc qdisc del dev {interface} root"
    
    # Execute setup commands
    for cmd in setup_cmds:
        stdin, stdout, stderr = ssh.exec_command(cmd)
        stdout.read()
    
    # Wait for the specified duration
    time.sleep(duration)
    
    # Remove the bandwidth limitation
    stdin, stdout, stderr = ssh.exec_command(remove_cmd)
    stdout.read()

def main():
    nodes = {
        # 'k8s-master': {'limit': '50%', 'ip': '_ip address__', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
        # 'k8s-worker-1': {'limit': '50%', 'ip': '172.26.128.30', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
        'k8s-worker-2': {'limit': '95%', 'ip': '172.26.132.91', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'}, # with server pod

    }
    duration = 60  # Duration for bandwidth limit in seconds
    
    for node, details in nodes.items():
        limit_percent = int(details['limit'].strip('%'))
        
        # Establish SSH connection using SSH keys and a context manager
        with paramiko.SSHClient() as ssh:
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.load_system_host_keys()  # Optional: load system host keys if needed
            ssh.connect(details['ip'], username=details['username'], key_filename=details['key_path'])
            
            print(f"Limiting bandwidth on {node} to {details['limit']}")
            limit_bandwidth(ssh, limit_percent, duration)
            print(f"Bandwidth limit removed from {node}")

if __name__ == "__main__":
    main()


Limiting bandwidth on k8s-worker-2 to 95%
Bandwidth limit removed from k8s-worker-2


In [ ]:
import paramiko
# import numpy as np

# modify the outbound traffic scheduler / egress qdisc
# injecting the trasmitting latency for the linux kerner scheduler when engressing packages 

def apply_latency_to_node(node_ip, username, key_path, interface, latency, varied_delay):
    """Apply latency to a network interface on a node using SSH with a private key."""
    #- Delay of 300ms and random 50ms uniform variation with correlation value 25% (since network delays are not completely random)
    # ----before using 'change', be sure there's already a 'add' rule in each node. Eg, Add a 0ms delay rule first
    # sudo  tc qdisc add dev eth0 root netem delay 300ms 50ms 25%
    
    command_clear = f"sudo tc qdisc del dev {interface} root" # delete all existing rules first and then add, otherwise the add rules may fail.
    command_add = f"sudo tc qdisc add dev {interface} root netem delay {latency}ms {varied_delay}ms 25%" 
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    client.load_system_host_keys()
    try:
        client.connect(node_ip, username=username, key_filename=key_path)
        stdin_clear, stdout_clear, stderr_clear = client.exec_command(command_clear)
        stdin_add, stdout_add, stderr_add = client.exec_command(command_add)

        print(stdout_add.read().decode())
    except Exception as e:
        print(f"Failed to apply latency to {node_ip}: {e}")
    finally:
        client.close()

def automate_latency_injection(node_details, latency = 0,  interface='eth0', varied_delay=0): # by default, set injected latency to 0ms
    """
    Apply latency to worker nodes using SSH keys for authentication.
    interface: Network interface to apply latency on.
    key_path: Path to the private SSH key.
    """

    for worker_name, worker_details in node_details.items():
        print(f"Applying {latency}ms latency with {varied_delay}ms varied delay to {worker_name}")
        apply_latency_to_node(worker_details['ip'], worker_details['username'], worker_details['key_path'], interface, latency, varied_delay)


#node_details: Dictionary with node names as keys and details as values (IP, username, key path).
node_details = {
    'k8s-worker-2': {'ip': '172.26.132.91', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-6': {'ip': '172.26.133.55', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-7': {'ip': '172.26.130.22', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-8': {'ip': '172.26.130.82', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-9': {'ip': '172.26.133.118', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},

    
    # 'k8s-worker-1': {'ip': '172.26.128.30', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-4': {'ip': '172.26.132.241', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    # 'k8s-worker-5': {'ip': '172.26.132.142', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'}
    
    # Add more nodes as needed
}


'''
Steps of traffic control for the tc commands:
(1) 'delete' all existing rules:  sudo tc qdisc del dev eth0 root
(2) 'add' delay : sudo tc qdisc add dev eth0 root netem delay 20ms
(3) 'change' delay :  sudo tc qdisc change dev eth0 root netem delay 300ms 30ms 25%
(4) 'show' the existing rules: sudo tc qdisc show  dev eth0
'''

automate_latency_injection(node_details, latency = 0, varied_delay=0)
